In [18]:
import tweepy
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
# from google.cloud import bigquery
import json

client = MongoClient('localhost', 27017)


# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
# bq_client = bigquery.Client()

db = client.worldcup

load_dotenv()

tweepy_client = tweepy.Client(os.getenv('BEARER_TOKEN'), wait_on_rate_limit=True)



In [25]:

query = '#ENG #FRA -is:retweet'

counts = client.get_recent_tweets_count(query=query, granularity='day')

for count in counts.data:
    print(count)

TypeError: 'Database' object is not callable

In [24]:
query = '#ARG #HRV #ARGHRV #HRVARG #Qatar2022 -is:retweet'

paginator = tweepy.Paginator(tweepy_client.search_recent_tweets, 
                        query=query,
                        tweet_fields = ['author_id', 'created_at', 'source', 'lang', 'geo', 'public_metrics', 'entities'], 
                        expansions = ['geo.place_id', 'author_id'], 
                        start_time = '2022-12-12T00:00:00.000Z',
                        end_time = '2022-12-13T23:00:00.000Z'
                        )
                        # user_fields = ['name','username','location','verified'], 
                        # place_fields = ['country','country_code'],

for page in paginator:
    
    users = {u["id"]: u for u in page.includes['users']}

    for tweet in page.data:

        if users[tweet.author_id]:
            name = users[tweet.author_id].name
            username = users[tweet.author_id].username

        person=[]
        for annotation in tweet.entities['annotations']:
            if annotation['type'] == 'Person':
                person.append(annotation['normalized_text'])

        tweet_content = {
            "tweet_id": tweet.id,
            "author_id": tweet.author_id,
            "author_name": name,
            "username": username,
            "created_at": tweet.created_at,
            "source": tweet.source,
            "lang": tweet.lang,
            "geo": tweet.geo,
            "retweet": tweet.public_metrics['retweet_count'],
            "reply": tweet.public_metrics['reply_count'],
            "like": tweet.public_metrics['like_count'],
            "quote": tweet.public_metrics['quote_count'],
            "hashtags": tweet.entities['hashtags'],
            # "urls": tweet.entities['urls'],
            "mention_person": ",".join(person),
            "text": tweet.text
        }
        
        db.ARGHRV.insert_one(tweet_content)

        break
    # print(tweet[0].entities['hashtags'])
    break
# for tweet in tweets.flatten():
#     try:
#         print("1. ",tweet.data['entities'])
#         print("2. ",tweet.includes['name'])
#         for key, value in tweet.data.items():
#             print (key, value)
#         # db.a.insert_one(tweet)
#         break
#     except ValueError as e:
#         print(e)
        

In [ ]:
search_query = "#JPN #HRV"
no_of_tweets = 1

tweets = tweepy_client.search_recent_tweets(query=search_query)


# i=0
for tweet in tweets:
    try:
        # content = {
        #     "Id": tweet.id,
        #     "Location": tweet.user.location,
        #     "Date_Created": tweet.created_at,
        #     "Tweet": tweet.text,
        #     "Reply_to": tweet.in_reply_to_user_id,
        #     "Lang": tweet.lang,
        #     "No_of_Likes": tweet.public_metrics.like_count,
        #     "No_of_Replies": tweet.public_metrics.reply_count,
        #     "No_of_retweets": tweet.public_metrics.retweet_count,
        #     "Source_of_Tweet": tweet.source
        # }
        print(tweet)
        print(type(tweet))
        break
        # db.t.insert_one(tweet)
        print(i)
    except ValueError as e:
        print(e)

In [17]:
eng_vs_fra = bq_client.get_table('tecky-capstone-project.worldcup.eng_fra')

x = db.ef.find()
for row in x:
    row['_id'] = str(row['_id'])
    row['Date_Created'] = str(row['Date_Created'])
    bq_client.insert_rows_json(_vs_hrv,[row])

NameError: name 'bq_client' is not defined